In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
from typing import List, Dict, Union, Tuple

import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import (
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM
)
from datasets import Dataset, DatasetDict, list_metrics, load_metric, load_from_disk


In [6]:
base_model = "microsoft/DialoGPT-small"
data_dir = "../data/empatheticdialogues"
output_dir = "../models/empathetic-DialoGPT-small"
batch_size = 2
logging_dir = output_dir
prediction_loss_only = True

In [68]:
def find_data_filepaths(data_dir):
    data_filepaths = list(Path(data_dir).glob("*.csv"))
    return data_filepaths


def load_data(data_dir):
    data = {}
    data_filepaths = find_data_filepaths(data_dir)
    for data_filepath in data_filepaths:
        data_name = data_filepath.stem
        data[data_name] = pd.read_csv(data_filepath, encoding="utf-8", on_bad_lines='skip')
    return data


def create_datasets(data, eos_token="<|endofsentence|>"):
    datasets = {}
    for name, df in data.items():
        grouped = df[["conv_id", "prompt", "utterance"]].groupby("conv_id")["utterance"]
        concat_text = grouped.transform(lambda x: eos_token.join(x))
        datasets[name] = Dataset.from_dict({"text": concat_text.unique()})
    return datasets

In [69]:
data = load_data(data_dir)
data["test"]

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
0,hit:0_conv:0,1,guilty,I felt guilty when I was driving home one nigh...,0,Yeah about 10 years ago I had a horrifying exp...,2|2|5_5|5|5,NaN
1,hit:0_conv:0,3,guilty,I felt guilty when I was driving home one nigh...,0,No I wasn't hit. It turned out they were drunk...,2|2|5_5|5|5,NaN
2,hit:0_conv:0,5,guilty,I felt guilty when I was driving home one nigh...,0,I don't know I was new to driving and hadn't e...,2|2|5_5|5|5,NaN
3,hit:34_conv:69,1,caring,My mother stopped by my house one day and said...,45,Well_comma_ can you tell me about your experie...,5|5|5_3|2|2,NaN
4,hit:34_conv:69,3,caring,My mother stopped by my house one day and said...,45,Oh my goodness_comma_ that's very scary! I hop...,5|5|5_3|2|2,NaN
...,...,...,...,...,...,...,...,...
5696,hit:12413_conv:24826,3,grateful,I'm glad that life is being good to me,437,Glad you think so as well!,5|5|5_4|3|4,NaN
5697,hit:12416_conv:24832,1,disgusted,I saw a huge cockroach outside my house today....,481,I saw a huge cockroach outside my house today!,5|5|5_4|3|4,NaN
5698,hit:12416_conv:24832,3,disgusted,I saw a huge cockroach outside my house today....,481,Not yet since it's the weekend. We live in Tex...,5|5|5_4|3|4,NaN
5699,hit:12423_conv:24847,1,anxious,I have a big test on Monday. I am so nervous_c...,481,I have a big test on Monday_comma_ I am so ner...,5|5|5_5|5|5,NaN


In [72]:
datasets = create_datasets(data)
print(datasets["test"].shape)
datasets["test"]

(2541, 1)


Dataset({
    features: ['text'],
    num_rows: 2541
})

In [101]:
def preprocess_function(tokenizer, text_column="text", max_length=256): 
    def _tokenize(examples):
        sanitized_text = [v.replace("_comma_", ",") for k, v in examples.items()]
        tokenized = tokenizer(
            sanitized_text,
            padding="max_length",
            max_length=max_length,
        )
        examples["input_ids"] = tokenized["input_ids"]
        return examples
    return _tokenize


def preprocess_datasets(datasets, tokenizer, text_column="text", max_length=256):
    columns = lambda d: d.features.keys()
    return {k: v.map(preprocess_function(tokenizer, text_column, max_length), remove_columns=columns(v)) for k, v in datasets.items()}
    

In [97]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [102]:
preprocessed_datasets = preprocess_datasets(datasets, tokenizer)

Parameter 'remove_columns'=dict_keys(['text']) of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2541 [00:00<?, ?ex/s]

  0%|          | 0/2763 [00:00<?, ?ex/s]

  0%|          | 0/17839 [00:00<?, ?ex/s]

In [103]:
preprocessed_datasets

{'test': Dataset({
     features: ['input_ids'],
     num_rows: 2541
 }),
 'valid': Dataset({
     features: ['input_ids'],
     num_rows: 2763
 }),
 'train': Dataset({
     features: ['input_ids'],
     num_rows: 17839
 })}